In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\hp\OneDrive\Desktop\csv_dataframe - Sheet1.csv")

In [3]:
df

,Course ID,Section,Teacher ID,Credit Hours,Semester
0,CE413,A,Mr. Salman Saeed,3,8
1,CE413,B,Mr. Salman Saeed,3,8
2,CS439,A,Dr. Farhan Khan,3,8
3,CS440,B,Dr. Farhan Khan,3,8
4,CS441,C,Dr. Farhan Khan,3,8
...,...,...,...,...,...
98,CS373,A,Dr. Fahad Bin Muslim,3,6
99,CS374,B,Dr. Fahad Bin Muslim,3,6
100,CS375,C,Dr. Fahad Bin Muslim,3,6
101,CS376,D,Dr. Fahad Bin Muslim,3,6


In [ ]:
# Python3 program to create target string, starting from 
# random string using Genetic Algorithm 
  
import random 
from collections import Counter

# Number of individuals in each generation 
POPULATION_SIZE = 100
  
  
class Individual(object): 
    ''' 
    Class representing individual in population 
    '''
    def __init__(self, chromosome): 
        self.chromosome = chromosome  
        self.fitness = self.cal_fitness() 

    @classmethod
    def create_gnome(self): 
        ''' 
        create chromosome or string of genes 
        '''
        
        week = []
        for i in range(50):
            lecture_hall = []
            for j in range(0,8):
                    lecture_hall.append(random.randint(0, 103))
            week.append(lecture_hall)
            
        return week
  
    def mate(self, par2): 
        ''' 
        Perform mating and produce new offspring 
        '''
  
        # chromosome for offspring 
        child_chromosome = [] 
        for gp1, gp2 in zip(self.chromosome, par2.chromosome):     
  
            # random probability   
            prob = random.random() 
  
            # if prob is less than 0.45, insert gene 
            # from parent 1  
            if prob < 0.45: 
                child_chromosome.append(gp1) 
  
            # if prob is between 0.45 and 0.90, insert 
            # gene from parent 2 
            elif prob < 0.90: 
                child_chromosome.append(gp2) 
  
            # otherwise insert random gene(mutate),  
            # for maintaining diversity 
            else: 
                lecture_hall = []
                for j in range(0,8):
                    lecture_hall.append(random.randint(0, 103))
                child_chromosome.append(lecture_hall) 
  
        # create new Individual(offspring) using  
        # generated chromosome for offspring 
        return Individual(child_chromosome) 
  
    def cal_fitness(self):
        
        # check for unique classes
        fitness  = 0 
        ctr = 0
        for chromosom in self.chromosome:
            counts = Counter(chromosom)
            fitness += sum(count for count in counts.values() if count > 1)
            
            
        for i in range(0,10):
            curr_chrom = self.chromosome[0]
            for j in range(0,len(self.chromosome)):
                for k in range (0,50,5):
                    if curr_chrom[j] == 
            
            
            
        
            
        return fitness
            
        

In [13]:
def main(): 
    global POPULATION_SIZE 
  
    #current generation 
    generation = 1
  
    found = False
    population = [] 
  
    # create initial population 
    for _ in range(POPULATION_SIZE): 
                gnome = Individual.create_gnome() 
                population.append(Individual(gnome)) 
  
    while not found: 
  
        # sort the population in increasing order of fitness score 
        population = sorted(population, key = lambda x:x.fitness) 
  
        # if the individual having lowest fitness score ie.  
        # 0 then we know that we have reached to the target 
        # and break the loop 
        if population[0].fitness <= 0: 
            found = True
            break
  
        # Otherwise generate new offsprings for new generation 
        new_generation = [] 
  
        # Perform Elitism, that mean 10% of fittest population 
        # goes to the next generation 
        s = int((10*POPULATION_SIZE)/100) 
        new_generation.extend(population[:s]) 
  
        # From 50% of fittest population, Individuals  
        # will mate to produce offspring 
        s = int((90*POPULATION_SIZE)/100) 
        for _ in range(s): 
            parent1 = random.choice(population[:50]) 
            parent2 = random.choice(population[:50]) 
            child = parent1.mate(parent2) 
            new_generation.append(child) 
  
        population = new_generation 
        print("Generation: {}\tString: {}\tFitness: {}",generation, population[0].fitness)
  
        generation += 1
  
      
    print("Generation: {}\tString: {}\tFitness: {}",generation, population[0].fitness)
    print(population[0].fitness)
    print(population[0].chromosome)
  
if __name__ == '__main__': 
    main() 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7